<a href="https://colab.research.google.com/github/jben-hun/colab_notebooks/blob/master/youtube_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip -q install aniso8601

import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
from datetime import timedelta
from aniso8601.duration import parse_duration
from google.colab import drive

PAGE_SIZE = 50

drive.mount("/content/drive")

# OAuth2 authentication

In [ ]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "/content/drive/My Drive/client_secret_1045491054300-fgaeppirms6uh6vbtto4jlo1fk9le2q1.apps.googleusercontent.com.json"

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

# Playlist duration calculation

In [ ]:
playListId = "PLeSiDiw7cGpMgGjxs1_GW7l1x8D_4x9lj"
playListId = "PLeSiDiw7cGpM6uPHJl9bIQeL2hpz-aEuC"

# get video identifiers from playlist identifier (PAGE_SIZE at a time)

videoIds = []
query = dict(
    part="contentDetails", maxResults=PAGE_SIZE, playlistId=playListId)
while True:
  request = youtube.playlistItems().list(**query)
  r = request.execute()
  videoIds += [i["contentDetails"]["videoId"] for i in r["items"]]
  if "nextPageToken" in r:
    query["pageToken"] = r["nextPageToken"]
  else:
    break

# get video metadata by video identifiers (PAGE_SIZE at a time)

durations = []
while videoIds:
  idList = videoIds[:PAGE_SIZE]
  videoIds = videoIds[PAGE_SIZE:]
  request = youtube.videos().list(
    part="contentDetails", id=",".join(idList))
  r = request.execute()
  durations += [i["contentDetails"]["duration"] for i in r["items"]]

# compute playtime statistics

seconds = tuple(parse_duration(d).total_seconds() for d in durations)
sumSeconds = sum(seconds)
lengthSeconds = len(seconds)
meanSeconds = sumSeconds / lengthSeconds
print(f"https://www.youtube.com/playlist?list={playListId}")
pd.DataFrame({
  "": dict(
    videos=lengthSeconds,
    mean_playtime=timedelta(seconds=meanSeconds),
    total_playtime=timedelta(seconds=sumSeconds))})

https://www.youtube.com/playlist?list=PLeSiDiw7cGpM6uPHJl9bIQeL2hpz-aEuC


,
mean_playtime,1:07:07.666667
total_playtime,6:42:46
videos,6
